In [13]:
import os, sys, glob, random
import numpy as np
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score, confusion_matrix, brier_score_loss
from hyperopt import hp, tpe, Trials
from hyperopt.fmin import fmin
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

import compare_auc_delong_xu

np.random.seed(529)
scaler = StandardScaler()

In [14]:
def getstats(y_true,y_pred,y_prob):
    rocauc = roc_auc_score(y_true,y_prob[:,1])
    brsc = round(brier_score_loss(y_true, y_prob[:,1]),2)
    prauc = round(average_precision_score(y_true,y_prob[:,1]),2)
    TN, FP, FN, TP = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    pos_pred_val = round(TP/ (TP+FP),2)
    neg_pred_val = round(TN/ (TN+FN),2)
    stats = [rocauc, prauc, brsc, pos_pred_val, neg_pred_val]
    return stats

def getAUC_CI(labels,scores):
    alpha = 0.95
    aucs = roc_auc_score(labels,scores)
    auc_delong, variances = compare_auc_delong_xu.delong_roc_variance(labels,scores)
    auc_std = np.sqrt(variances)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)
    ci = stats.norm.ppf(lower_upper_q,loc=auc_delong,scale=auc_std)
    return auc_delong, variances, ci

In [15]:
basefile = pd.read_csv('/data/project/bhattlab/boddugit/datafolder/COPD_P1_P2_P3_10.2021.csv',low_memory=False)
basefile = basefile[['sid','FEV1_FVC_post_P1']]
basefile['COPD_P1'] = pd.cut(basefile['FEV1_FVC_post_P1'], bins=[0, 0.7, float('Inf')], labels=[1, 0])
basefile = basefile[['sid','COPD_P1']]
basefile = basefile.dropna()

# merging the base file with radiomics feature file
df1 = pd.read_csv('/data/project/bhattlab/appr/PyRadiomics_features_set/Phase1/phase1_data.csv')
df1base = pd.merge(df1,basefile,on='sid')
print('Size after merging with radiomics feature file',df1base.shape)
print('Class Distribution\n',df1base['COPD_P1_x'].value_counts())

#saving the file for Table 1 and Table 3 measurements
df1base.to_csv('standard-dose-dataframe-radiology.csv',index=False)

X1 = df1base.iloc[:,1:124].astype('float64')
y1 = df1base['COPD_P1_x']

Size after merging with radiomics feature file (8878, 126)
Class Distribution
 0    4895
1    3983
Name: COPD_P1_x, dtype: int64


In [16]:
seed = 529
Xt, Xv, yt, yv = train_test_split(X1, y1, test_size = 0.2, random_state = seed, stratify=y1)
Xt, Xv = np.array(Xt), np.array(Xv)
print(Xt.shape,Xv.shape)
Xtdemo, Xtqct, Xtrad = Xt[:,0:6], Xt[:,6:7], Xt[:,8:]
Xvdemo, Xvqct, Xvrad = Xv[:,0:6], Xv[:,6:7], Xv[:,8:]
Xtrad, Xvrad = StandardScaler().fit_transform(Xtrad), StandardScaler().fit_transform(Xvrad)
Xtcomb = np.concatenate((Xtdemo,Xtqct,Xtrad),axis=1)
Xvcomb = np.concatenate((Xvdemo,Xvqct,Xvrad),axis=1)
print(Xtrad.shape,Xtcomb.shape)

clfdemo = CatBoostClassifier(loss_function='CrossEntropy')
clfdemo.fit(Xtdemo,yt)
y_pred_demo = clfdemo.predict(Xvdemo)
y_prob_demo = clfdemo.predict_proba(Xvdemo)
demostats = getstats(yv,y_pred_demo,y_prob_demo)

clfqct = CatBoostClassifier(loss_function='CrossEntropy')
clfqct.fit(Xtqct,yt)
y_pred_qct = clfqct.predict(Xvqct)
y_prob_qct = clfqct.predict_proba(Xvqct)
qctstats = getstats(yv,y_pred_qct,y_prob_qct)

clfrad = CatBoostClassifier(loss_function='CrossEntropy')
clfrad.fit(Xtrad,yt)
y_pred_rad = clfrad.predict(Xvrad)
y_prob_rad = clfrad.predict_proba(Xvrad)
radstats = getstats(yv,y_pred_rad,y_prob_rad)

clfcomb = CatBoostClassifier(loss_function='CrossEntropy')
clfcomb.fit(Xtcomb,yt)
y_pred_comb = clfcomb.predict(Xvcomb)
y_prob_comb = clfcomb.predict_proba(Xvcomb)
combstats = getstats(yv,y_pred_comb,y_prob_comb)

print()
print('rocauc', 'prauc', 'sensitivity', 'specificity', 'pos_pred_val', 'neg_pred_val')
print('demographics -> AUC, PRAUC, BRIER, PPV, NPV', demostats)
print('CT emphysema -> AUC, PRAUC, BRIER, PPV, NPV', qctstats)
print('CT Radiomics -> AUC, PRAUC, BRIER, PPV, NPV', combstats)
print('Combined     -> AUC, PRAUC, BRIER, PPV, NPV', combstats)

(7102, 123) (1776, 123)
(7102, 115) (7102, 122)
0:	learn: 0.6865915	total: 1.83ms	remaining: 1.83s
1:	learn: 0.6808940	total: 3.73ms	remaining: 1.86s
2:	learn: 0.6754277	total: 5.42ms	remaining: 1.8s
3:	learn: 0.6699743	total: 7.49ms	remaining: 1.86s
4:	learn: 0.6649794	total: 9.25ms	remaining: 1.84s
5:	learn: 0.6599745	total: 10.8ms	remaining: 1.8s
6:	learn: 0.6558292	total: 12.9ms	remaining: 1.83s
7:	learn: 0.6518923	total: 14.6ms	remaining: 1.8s
8:	learn: 0.6481819	total: 16.2ms	remaining: 1.78s
9:	learn: 0.6442183	total: 18.8ms	remaining: 1.86s
10:	learn: 0.6409364	total: 20.6ms	remaining: 1.85s
11:	learn: 0.6373906	total: 22.3ms	remaining: 1.84s
12:	learn: 0.6343338	total: 24.2ms	remaining: 1.83s
13:	learn: 0.6316490	total: 26.2ms	remaining: 1.84s
14:	learn: 0.6285804	total: 28.1ms	remaining: 1.85s
15:	learn: 0.6257215	total: 29.8ms	remaining: 1.83s
16:	learn: 0.6229229	total: 31.4ms	remaining: 1.82s
17:	learn: 0.6202443	total: 33.7ms	remaining: 1.84s
18:	learn: 0.6179139	total: 3

176:	learn: 0.5504810	total: 361ms	remaining: 1.68s
177:	learn: 0.5503624	total: 363ms	remaining: 1.68s
178:	learn: 0.5502391	total: 365ms	remaining: 1.67s
179:	learn: 0.5501401	total: 366ms	remaining: 1.67s
180:	learn: 0.5500326	total: 368ms	remaining: 1.67s
181:	learn: 0.5499554	total: 370ms	remaining: 1.66s
182:	learn: 0.5498927	total: 372ms	remaining: 1.66s
183:	learn: 0.5497858	total: 373ms	remaining: 1.66s
184:	learn: 0.5497771	total: 374ms	remaining: 1.65s
185:	learn: 0.5497285	total: 376ms	remaining: 1.65s
186:	learn: 0.5495714	total: 378ms	remaining: 1.64s
187:	learn: 0.5495069	total: 380ms	remaining: 1.64s
188:	learn: 0.5493920	total: 381ms	remaining: 1.64s
189:	learn: 0.5493241	total: 383ms	remaining: 1.63s
190:	learn: 0.5491617	total: 385ms	remaining: 1.63s
191:	learn: 0.5490781	total: 386ms	remaining: 1.63s
192:	learn: 0.5489487	total: 388ms	remaining: 1.62s
193:	learn: 0.5487474	total: 390ms	remaining: 1.62s
194:	learn: 0.5486326	total: 392ms	remaining: 1.62s
195:	learn: 

360:	learn: 0.5305601	total: 757ms	remaining: 1.34s
361:	learn: 0.5304648	total: 759ms	remaining: 1.34s
362:	learn: 0.5303750	total: 761ms	remaining: 1.33s
363:	learn: 0.5302004	total: 763ms	remaining: 1.33s
364:	learn: 0.5300686	total: 765ms	remaining: 1.33s
365:	learn: 0.5299503	total: 767ms	remaining: 1.33s
366:	learn: 0.5298430	total: 769ms	remaining: 1.33s
367:	learn: 0.5297767	total: 771ms	remaining: 1.32s
368:	learn: 0.5296282	total: 772ms	remaining: 1.32s
369:	learn: 0.5294931	total: 774ms	remaining: 1.32s
370:	learn: 0.5294461	total: 776ms	remaining: 1.31s
371:	learn: 0.5292807	total: 778ms	remaining: 1.31s
372:	learn: 0.5291765	total: 779ms	remaining: 1.31s
373:	learn: 0.5291103	total: 781ms	remaining: 1.31s
374:	learn: 0.5289836	total: 783ms	remaining: 1.3s
375:	learn: 0.5288535	total: 785ms	remaining: 1.3s
376:	learn: 0.5287292	total: 787ms	remaining: 1.3s
377:	learn: 0.5286102	total: 788ms	remaining: 1.3s
378:	learn: 0.5284892	total: 790ms	remaining: 1.29s
379:	learn: 0.52

550:	learn: 0.5096713	total: 1.15s	remaining: 940ms
551:	learn: 0.5095862	total: 1.16s	remaining: 938ms
552:	learn: 0.5094733	total: 1.16s	remaining: 935ms
553:	learn: 0.5094269	total: 1.16s	remaining: 934ms
554:	learn: 0.5092898	total: 1.16s	remaining: 931ms
555:	learn: 0.5092134	total: 1.16s	remaining: 929ms
556:	learn: 0.5091086	total: 1.17s	remaining: 927ms
557:	learn: 0.5090145	total: 1.17s	remaining: 924ms
558:	learn: 0.5089675	total: 1.17s	remaining: 922ms
559:	learn: 0.5088252	total: 1.17s	remaining: 920ms
560:	learn: 0.5087053	total: 1.17s	remaining: 917ms
561:	learn: 0.5085266	total: 1.17s	remaining: 915ms
562:	learn: 0.5083452	total: 1.18s	remaining: 913ms
563:	learn: 0.5082460	total: 1.18s	remaining: 910ms
564:	learn: 0.5081368	total: 1.18s	remaining: 908ms
565:	learn: 0.5080644	total: 1.18s	remaining: 906ms
566:	learn: 0.5079270	total: 1.18s	remaining: 904ms
567:	learn: 0.5078329	total: 1.19s	remaining: 901ms
568:	learn: 0.5077221	total: 1.19s	remaining: 899ms
569:	learn: 

736:	learn: 0.4916837	total: 1.59s	remaining: 570ms
737:	learn: 0.4915796	total: 1.6s	remaining: 567ms
738:	learn: 0.4914582	total: 1.6s	remaining: 565ms
739:	learn: 0.4913896	total: 1.6s	remaining: 563ms
740:	learn: 0.4913229	total: 1.6s	remaining: 560ms
741:	learn: 0.4912042	total: 1.6s	remaining: 558ms
742:	learn: 0.4911094	total: 1.61s	remaining: 556ms
743:	learn: 0.4910644	total: 1.61s	remaining: 554ms
744:	learn: 0.4910003	total: 1.61s	remaining: 551ms
745:	learn: 0.4908876	total: 1.61s	remaining: 549ms
746:	learn: 0.4908300	total: 1.61s	remaining: 547ms
747:	learn: 0.4907809	total: 1.62s	remaining: 545ms
748:	learn: 0.4907343	total: 1.62s	remaining: 543ms
749:	learn: 0.4906596	total: 1.62s	remaining: 541ms
750:	learn: 0.4905899	total: 1.62s	remaining: 538ms
751:	learn: 0.4904596	total: 1.63s	remaining: 536ms
752:	learn: 0.4903842	total: 1.63s	remaining: 534ms
753:	learn: 0.4903277	total: 1.63s	remaining: 531ms
754:	learn: 0.4901863	total: 1.63s	remaining: 529ms
755:	learn: 0.490

...

208:	learn: 0.3061958	total: 1.98s	remaining: 7.51s
209:	learn: 0.3059253	total: 1.99s	remaining: 7.5s
210:	learn: 0.3055912	total: 2s	remaining: 7.49s
211:	learn: 0.3053258	total: 2.02s	remaining: 7.5s
212:	learn: 0.3049520	total: 2.04s	remaining: 7.52s
213:	learn: 0.3047370	total: 2.04s	remaining: 7.51s
214:	learn: 0.3045082	total: 2.05s	remaining: 7.5s
215:	learn: 0.3042555	total: 2.06s	remaining: 7.49s
216:	learn: 0.3040272	total: 2.07s	remaining: 7.47s
217:	learn: 0.3037756	total: 2.08s	remaining: 7.46s
218:	learn: 0.3035517	total: 2.09s	remaining: 7.46s
219:	learn: 0.3031694	total: 2.1s	remaining: 7.45s
220:	learn: 0.3029225	total: 2.11s	remaining: 7.44s
221:	learn: 0.3026647	total: 2.12s	remaining: 7.43s
222:	learn: 0.3023574	total: 2.13s	remaining: 7.42s
223:	learn: 0.3020605	total: 2.14s	remaining: 7.41s
224:	learn: 0.3018093	total: 2.15s	remaining: 7.4s
225:	learn: 0.3015580	total: 2.16s	remaining: 7.39s
226:	learn: 0.3012846	total: 2.17s	remaining: 7.37s
227:	learn: 0.301035

388:	learn: 0.2579255	total: 3.64s	remaining: 5.72s
389:	learn: 0.2576684	total: 3.65s	remaining: 5.71s
390:	learn: 0.2574233	total: 3.66s	remaining: 5.7s
391:	learn: 0.2571933	total: 3.67s	remaining: 5.69s
392:	learn: 0.2569289	total: 3.68s	remaining: 5.68s
393:	learn: 0.2565569	total: 3.68s	remaining: 5.67s
394:	learn: 0.2562992	total: 3.69s	remaining: 5.66s
395:	learn: 0.2561041	total: 3.7s	remaining: 5.65s
396:	learn: 0.2557928	total: 3.71s	remaining: 5.64s
397:	learn: 0.2555598	total: 3.72s	remaining: 5.63s
398:	learn: 0.2552889	total: 3.73s	remaining: 5.62s
399:	learn: 0.2550898	total: 3.74s	remaining: 5.61s
400:	learn: 0.2549575	total: 3.75s	remaining: 5.6s
401:	learn: 0.2547070	total: 3.76s	remaining: 5.59s
402:	learn: 0.2544463	total: 3.76s	remaining: 5.58s
403:	learn: 0.2541649	total: 3.77s	remaining: 5.57s
404:	learn: 0.2538215	total: 3.78s	remaining: 5.56s
405:	learn: 0.2535313	total: 3.79s	remaining: 5.55s
406:	learn: 0.2532483	total: 3.8s	remaining: 5.54s
407:	learn: 0.25

557:	learn: 0.2180936	total: 5.29s	remaining: 4.19s
558:	learn: 0.2178958	total: 5.3s	remaining: 4.18s
559:	learn: 0.2176283	total: 5.31s	remaining: 4.17s
560:	learn: 0.2174433	total: 5.32s	remaining: 4.16s
561:	learn: 0.2172669	total: 5.33s	remaining: 4.15s
562:	learn: 0.2170906	total: 5.34s	remaining: 4.14s
563:	learn: 0.2169009	total: 5.35s	remaining: 4.13s
564:	learn: 0.2166223	total: 5.36s	remaining: 4.13s
565:	learn: 0.2162984	total: 5.37s	remaining: 4.12s
566:	learn: 0.2161879	total: 5.38s	remaining: 4.11s
567:	learn: 0.2160701	total: 5.39s	remaining: 4.1s
568:	learn: 0.2159703	total: 5.4s	remaining: 4.09s
569:	learn: 0.2159086	total: 5.4s	remaining: 4.08s
570:	learn: 0.2157513	total: 5.41s	remaining: 4.07s
571:	learn: 0.2154696	total: 5.42s	remaining: 4.06s
572:	learn: 0.2152111	total: 5.43s	remaining: 4.05s
573:	learn: 0.2149322	total: 5.44s	remaining: 4.04s
574:	learn: 0.2147907	total: 5.45s	remaining: 4.03s
575:	learn: 0.2146500	total: 5.46s	remaining: 4.01s
576:	learn: 0.21

734:	learn: 0.1830226	total: 6.94s	remaining: 2.5s
735:	learn: 0.1827911	total: 6.95s	remaining: 2.49s
736:	learn: 0.1826318	total: 6.96s	remaining: 2.48s
737:	learn: 0.1823970	total: 6.96s	remaining: 2.47s
738:	learn: 0.1822731	total: 6.97s	remaining: 2.46s
739:	learn: 0.1820033	total: 6.98s	remaining: 2.45s
740:	learn: 0.1818703	total: 6.99s	remaining: 2.44s
741:	learn: 0.1816741	total: 7s	remaining: 2.43s
742:	learn: 0.1815325	total: 7.01s	remaining: 2.42s
743:	learn: 0.1813509	total: 7.04s	remaining: 2.42s
744:	learn: 0.1812411	total: 7.05s	remaining: 2.41s
745:	learn: 0.1810592	total: 7.07s	remaining: 2.4s
746:	learn: 0.1808541	total: 7.07s	remaining: 2.4s
747:	learn: 0.1806779	total: 7.08s	remaining: 2.38s
748:	learn: 0.1805260	total: 7.09s	remaining: 2.38s
749:	learn: 0.1804262	total: 7.1s	remaining: 2.37s
750:	learn: 0.1802078	total: 7.11s	remaining: 2.36s
751:	learn: 0.1799704	total: 7.12s	remaining: 2.35s
752:	learn: 0.1798283	total: 7.13s	remaining: 2.34s
753:	learn: 0.17967

908:	learn: 0.1540639	total: 8.61s	remaining: 862ms
909:	learn: 0.1539598	total: 8.62s	remaining: 853ms
910:	learn: 0.1538362	total: 8.63s	remaining: 843ms
911:	learn: 0.1537112	total: 8.64s	remaining: 834ms
912:	learn: 0.1535627	total: 8.65s	remaining: 824ms
913:	learn: 0.1534099	total: 8.66s	remaining: 815ms
914:	learn: 0.1532434	total: 8.66s	remaining: 805ms
915:	learn: 0.1531514	total: 8.67s	remaining: 795ms
916:	learn: 0.1530495	total: 8.68s	remaining: 786ms
917:	learn: 0.1528821	total: 8.69s	remaining: 776ms
918:	learn: 0.1527533	total: 8.7s	remaining: 767ms
919:	learn: 0.1526402	total: 8.71s	remaining: 757ms
920:	learn: 0.1525312	total: 8.72s	remaining: 748ms
921:	learn: 0.1523439	total: 8.73s	remaining: 738ms
922:	learn: 0.1521361	total: 8.74s	remaining: 729ms
923:	learn: 0.1520012	total: 8.76s	remaining: 721ms
924:	learn: 0.1518799	total: 8.77s	remaining: 711ms
925:	learn: 0.1517470	total: 8.78s	remaining: 702ms
926:	learn: 0.1515968	total: 8.79s	remaining: 692ms
927:	learn: 0

In [17]:
aucdemo, aucvardemo, auccidemo = getAUC_CI(yv,y_prob_demo[:,1])
aucqct, aucvarqct, aucciqct = getAUC_CI(yv,y_prob_qct[:,1])
aucrad, aucvarrad, auccirad = getAUC_CI(yv,y_prob_rad[:,1])
auccomb, aucvarcomb, auccicomb = getAUC_CI(yv,y_prob_comb[:,1])

print('demographics -> CI ',aucdemo,auccidemo)
print('CT emphysema -> CI ',aucqct,aucciqct)
print('CT radiomics -> CI ',aucrad, auccirad)
print('combined     => CI ',auccomb, auccicomb)

auc_rad_vs_demo = compare_auc_delong_xu.delong_roc_test(yv,y_prob_rad[:,1],y_prob_demo[:,1])
auc_rad_vs_qct = compare_auc_delong_xu.delong_roc_test(yv,y_prob_rad[:,1],y_prob_qct[:,1])
auc_rad_vs_comb = compare_auc_delong_xu.delong_roc_test(yv,y_prob_rad[:,1],y_prob_comb[:,1])
print('AUC Comparison - Demographics vs. Radiomics, DeLong p-value ', np.exp(auc_rad_vs_demo))
print('AUC Comparison - CT emphysema vs. Radiomics, DeLong p-value  ',np.exp(auc_rad_vs_qct))
print('AUC Comparison - Combined vs. Radiomics, DeLong p-value  ',np.exp(auc_rad_vs_comb))


standarddose_feature_imp = pd.DataFrame(sorted(zip(clfrad.feature_importances_,X1.columns[8:])), columns=['Value','Feature'])
standarddose_feature_imp = standarddose_feature_imp.sort_values(by="Value", ascending=False)
standarddose_feature_imp[0:10]

demographics -> CI  0.7384356300375642 [0.71542901 0.76144225]
CT emphysema -> CI  0.8213179402329728 [0.80147294 0.84116294]
CT radiomics -> CI  0.9017280070950436 [0.88729098 0.91616503]
combined     => CI  0.9086397791513888 [0.89490822 0.92237134]
AUC Comparison - Demographics vs. Radiomics, DeLong p-value  [[5.50079205e-19]]
AUC Comparison - CT emphysema vs. Radiomics, DeLong p-value   [[4.29837682e-10]]
AUC Comparison - Combined vs. Radiomics, DeLong p-value   [[0.15628968]]


,Value,Feature
114,8.525172,10Percentile
113,4.304300,TotalEnergy
112,4.295605,original_shape_Sphericity
111,3.111495,rightoriginal_shape_Sphericity
110,3.022179,RunEntropy
109,2.947082,Imc2
108,2.766349,original_shape_MinorAxisLength
107,2.478190,original_shape_SurfaceArea
106,2.309047,GrayLevelNonUniformityNormalized.1
105,2.250415,LowGrayLevelRunEmphasis
